In [1]:
import spacy
import requests
import datetime
import json
nlp = spacy.load('en_core_web_lg')

In [52]:
def print_intent(params):
    text = params[0]
    intent = params[1]
    predicate = params[2]
    objects = params[3]
    if not intent:
        print('No intent for "' + text + '"')
    else:
        print(text,'==>',intent,objects)

    print('-----------------')

## Let's get the root!

We use the **root** of the sentence/clause as the predicate in the subject-predicate-object structure.

In [58]:
def intent_from_root(text):
    doc = nlp(text)
    intent = None
    predicate = None
    objects = [ent.text for ent in doc.ents]
    
    predicate_taxonomy = {
        'directed':['direct','director'],
        'cast':['star','feature','cameo']
    }
        
    #get root of the clause
    for tok in doc:
        if tok.dep_ == "ROOT":
            predicate = tok

    #See if the intent is in the taxonomy
    for kind in predicate_taxonomy.keys():
        if predicate.lemma_ in predicate_taxonomy[kind]:
            intent = kind
            
    return [text,intent,predicate,objects]
            
print_intent(intent_from_root("the movie was directed by george clooney"))
print_intent(intent_from_root("movies directed by george clooney"))
print_intent(intent_from_root("movies by george clooney"))
print_intent(intent_from_root("directed by george clooney"))
print_intent(intent_from_root("george clooney director"))
print_intent(intent_from_root("george clooney directing with brad pitt"))
print_intent(intent_from_root("george clooney directed brad pitt"))
print_intent(intent_from_root("george clooney directed by brad pitt"))

print_intent(intent_from_root("The movie stars george clooney"))
print_intent(intent_from_root("starring george clooney"))
print_intent(intent_from_root("george clooney starred"))
print_intent(intent_from_root("george clooney star"))

print_intent(intent_from_root("george clooney and brad pitt"))
print_intent(intent_from_root("george clooney with brad pitt"))

the movie was directed by george clooney ==> directed ['george clooney']
-----------------
No intent for "movies directed by george clooney"
-----------------
No intent for "movies by george clooney"
-----------------
directed by george clooney ==> directed ['george clooney']
-----------------
george clooney director ==> directed ['george clooney']
-----------------
No intent for "george clooney directing with brad pitt"
-----------------
george clooney directed brad pitt ==> directed ['george clooney', 'brad pitt']
-----------------
No intent for "george clooney directed by brad pitt"
-----------------
The movie stars george clooney ==> cast ['george clooney']
-----------------
starring george clooney ==> cast ['george clooney']
-----------------
george clooney starred ==> cast ['george clooney']
-----------------
george clooney star ==> cast ['george clooney']
-----------------
No intent for "george clooney and brad pitt"
-----------------
No intent for "george clooney with brad pitt

In [67]:
def intent_from_root2(text):
    doc = nlp(text)
    intent = None
    predicate = None
    objects = [ent.text for ent in doc.ents]
    
    predicate_taxonomy = {
        'directed':['direct','director'],
        'cast':['star','feature','cameo']
    }
        
    #get root of the clause
    for tok in doc:
        print(tok.text,tok.dep_)
        if tok.dep_ == "ROOT":
            predicate = tok
    
    print(predicate,objects)
    
    #See if the intent is in the taxonomy
    for kind in predicate_taxonomy.keys():
        if predicate.lemma_ in predicate_taxonomy[kind]:
            intent = kind
            
    return [text,intent,predicate,objects]
            
print_intent(intent_from_root2("best george clooney movie"))
print_intent(intent_from_root2("best movie of 2012"))
print_intent(intent_from_root2("2012 worst"))
print_intent(intent_from_root2("Tom Hanks' best movie"))
print_intent(intent_from_root2("underrated tom hanks"))
print_intent(intent_from_root2("panned julia roberts"))
print_intent(intent_from_root2("popular julia roberts"))

best amod
george compound
clooney compound
movie ROOT
movie ['george clooney']
No intent for "best george clooney movie"
-----------------
best amod
movie ROOT
of prep
2012 pobj
movie ['2012']
No intent for "best movie of 2012"
-----------------
2012 nummod
worst ROOT
worst ['2012']
No intent for "2012 worst"
-----------------
Tom compound
Hanks poss
' case
best amod
movie ROOT
movie ["Tom Hanks'"]
No intent for "Tom Hanks' best movie"
-----------------
underrated amod
tom compound
hanks ROOT
hanks []
No intent for "underrated tom hanks"
-----------------
panned ROOT
julia compound
roberts dobj
panned []
No intent for "panned julia roberts"
-----------------
popular amod
julia compound
roberts ROOT
roberts []
No intent for "popular julia roberts"
-----------------
